In [ ]:
# 2003-2013 No license number
import pandas as pd
bk_res_old = pd.DataFrame()
for i in range(2003, 2014):
    bk = pd.read_excel(f'../files4Import/enerji-bakanligi/{i}.xls', header=2)
    bk = bk[bk['YAKIT CİNSİ'] == 'RES']
    bk_res_old = pd.concat([bk_res_old, bk])
bk_res_old.drop(columns=['SIRA NO','YAKIT CİNSİ'], inplace=True)
bk_res_old = bk_res_old.rename(columns={'  GEÇİCİ KABULTARİHİ': 'KABUL TARİHİ', 'İLAVE KURULU GÜÇ MWe': 'İLAVE KURULU GÜÇ (MWe)', 'ÜNİTE GÜCÜ MWe': 'ÜNİTE GÜCÜ (MWe)'})
bk_res_old

In [ ]:
import sympy as sp

def convert_to_float(text):
    text = str(text)
    # Replace all variables with symbols
    text = text.replace("x", "*")
    text = text.replace("X", "*")
    text = text.replace(",", ".")

    if text == "-" or text == "":
        return text
    # Convert the expression to a sympy expression
    expr = sp.sympify(text)
    
    # Evaluate the expression numerically
    result = sp.N(expr)
    result = float(result)
    return result

In [ ]:
import pandas as pd
bk_res = pd.DataFrame()
for i in range(2014, 2024):
    if i >= 2019:
        bk = pd.read_excel(f'../files4Import/enerji-bakanligi/{i}.xlsx', header=1)
    else:
        bk = pd.read_excel(f'../files4Import/enerji-bakanligi/{i}.xlsx', header=2)
        bk.rename(columns={'  GEÇİCİ KABUL TARİHİ': 'KABUL TARİHİ', 'GEÇİCİ KABUL TARİHİ': 'KABUL TARİHİ'}, inplace=True)
    if 'YAKIT CİNSİ' not in bk.columns:
        bk = bk[bk['KAYNAK'] == 'RES']
        bk.rename(columns={'KAYNAK': 'YAKIT CİNSİ'}, inplace=True)
        
    else:
        bk = bk[bk['YAKIT CİNSİ'] == 'RES']
        bk.rename(columns={'İLAVE KURULU GÜÇ MWe': 'İLAVE KURULU GÜÇ (MWe)', 'ÜNİTE GÜCÜ MWe': 'ÜNİTE GÜCÜ (MWe)',}, inplace=True)
    bk_res = pd.concat([bk_res, bk])
bk_res.drop(columns=['SIRA NO','YAKIT CİNSİ'], inplace=True)
bk_res

In [ ]:
from src.sshDb import ssh_db

# Start SSH tunnel and connect to DB
ssh_db.connect_ssh_db()
# Get table wf from database
wfPostgre = pd.read_sql_table('wf', ssh_db.engine)
wfPostgre = wfPostgre.loc[wfPostgre['version'] == wfPostgre['version'].max()]

ministryOfEnergy = pd.merge(wfPostgre[['id']], bk_res, right_on="LİSANS SAYISI", left_on=wfPostgre["Lisans No"], how="right", suffixes=('', ''))
ministryOfEnergy = pd.merge(ministryOfEnergy, bk_res_old, how='outer')

ministryOfEnergy = ministryOfEnergy.set_index('id')
ministryOfEnergy['LİSANS TARİHİ']= pd.to_datetime(ministryOfEnergy['LİSANS TARİHİ']) # Turn to UTC ???????


#ministryOfEnergy["Calculated ÜNİTE GÜCÜ (MWe)"] = ministryOfEnergy["ÜNİTE GÜCÜ (MWe)"].apply(convert_to_float)


ministryOfEnergy.to_sql('ministry_of_energy', ssh_db.engine, if_exists='replace', index=True)
ministryOfEnergy